In [2]:
# !pip install tabula-py


In [667]:
import os
import tabula
import pdfplumber
from pprint import pprint
import pandas as pd
import numpy as np
import zipfile

In [684]:
file_path = '../data/tianma.pdf'
# file_path = '../data/tianma_season_1.pdf'
# file_path = '../data/浦发银行半年报.PDF'
# pdf = pdfplumber.open(file_path)

In [675]:
def extratPDF(file_path, table_method='pdfplumber', text_method='pdfplumber'):
    '''先获取表格，然后处理句子
       table_method 可选 'tabula', 'pdfplumber', None
       text_method 可选 'pdfplumber', 'pdfminer', None
    '''
    if not os.path.isfile(file_path):
        print(f'file {file_path} not found')
        return None
    
    info = {'tables':[], 'text':[]}
    pdf = pdfplumber.open(file_path)   
    # 处理表格
    if table_method is None:
        pass
    elif table_method == 'pdfplumber':
        info['tables'] = extract_table_by_pdfplumber(pdf, text_tolerance=2)
    elif table_method == 'tabula':
        info['tables'] = extract_table_by_tabula(file_path)
    else:
        print(f'other table extract method {table_method} has not implement yet')
    if info['tables']:
        for i,table in enumerate(info['tables']):
            info['tables'][i] = process_raw_table(table)
            
    # 处理句子
    if text_method is None:
        pass
    elif text_method == 'pdfplumber':
        info['text'] = extract_text_by_pdfplumber(pdf)
    else:
        print(f'other text extract method {text_method} has not implement yet')
    if info['text']:
        info['text'] = process_raw_text(info['text'])
    
    return info

In [676]:
def process_raw_text(text, split=['。', '/n']):
    '''处理文本：切分句子和表格的行，存储到es等
    '''
    pass

In [686]:
def convert2csv(file_path, compress=False):
    '''提取pdf中的表格并保存在文件同名目录下
       每个 DataFrame 输出一个 csv 文件
    '''
    pdf_info = extratPDF(
        file_path, 
        table_method='pdfplumber', 
        text_method=None)
    tables = pdf_info['tables']
    
    csv_path = file_path.replace('.PDF','').replace('.pdf', '') 
    if compress:
        zip_file = zipfile.ZipFile(csv_path + '.zip', 'w')
    if not os.path.exists(csv_path):
        print(csv_path)
        os.makedirs(csv_path)
    for i, table in enumerate(tables):
        table.to_csv(f'{csv_path}{os.sep}{i+1}.csv', 
                     index=False, header=False,
                     encoding='gbk')
        if compress:
            zip_file.write(
                f'{csv_path}{os.sep}{i+1}.csv',
                f'{os.path.split(csv_path)[-1]}{os.sep}{i+1}.csv')
    if compress:
        zip_file.close()
        return csv_path + '.zip'
    return csv_path + os.sep

convert2csv(file_path,compress=True)

../data/tianma


'../data/tianma.zip'

In [678]:
('../data/tianma')

'../data/tianma'

In [679]:
def convert2excel(file_path, save_path=None):
    '''提取pdf中的表格并保存为 excel文件
       每个DataFrame一个Sheet
    '''
    pdf_info = extratPDF(
        file_path, 
        table_method='pdfplumber', 
        text_method=None)
    tables = pdf_info['tables']
    
    excel_path = file_path.replace('.PDF','').replace('.pdf', '')
    with pd.ExcelWriter(excel_path + '.xlsx') as excel:
        for i, table in enumerate(tables):
            table.to_excel(excel, f'Sheet{i+1}')
    return excel_path + '.xlsx'

In [680]:
%%time
convert2excel(file_path)

CPU times: user 2min 12s, sys: 4.31 s, total: 2min 16s
Wall time: 2min 28s


'../data/浦发银行半年报.xlsx'

In [626]:
%%time
def extract_text_by_pdfplumber(file, **kargs):
    # 处理文本，返回句子等
    if isinstance(file, str):
        pages = pdfplumber.open(file).pages
    if isinstance(file, pdfplumber.pdf.PDF):
        pages = file.pages
        
    text = [[]]
    for page in pages:
        x0,y0 = Decimal(0), Decimal(0)
        for char in page.chars:
            if (char['x0'] < x0-20) and (char['y0'] < y0-20):
                print('case 1', char['text'])
                x0, y0 = char['x0'], char['y0']
                text.append([char['text']])
            elif (char['x0'] > x0+1) and (char['y0'] > y0+1):
                x0, y0 = char['x0'], char['y0']
                text.append([char['text']])
                print('case 2', char['text'])
            else:
                text[-1].append(char['text'])
                # print(char['text'], end='')
        # raw_text = [char['text'] for char in page.chars]
        # text.extend(raw_text)
    return text

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 46.5 µs


In [488]:
%%time
def extract_table_by_tabula(file_path, page='all'):
    '''使用 tabula-java输出的json数据来提取表格'''
    tables = []
    try:
        tables = tabula.read_pdf(
            file_path,
            encoding='utf-8', 
            pages='all',
            multiple_tables=True
        )
    except Exception as e:
        print('Error in extract_table_by_tabula:', e)
    finally:
        return tables
    
tables_2 = extract_table_by_tabula(file_path)
print(len(tables_2));tables_2[-1]

29
CPU times: user 30 ms, sys: 40 ms, total: 70 ms
Wall time: 23.7 s


In [489]:
%%time
def extract_table_by_pdfplumber(file, text_tolerance=2):
    '''使用 pdfplumber 的两种分析模式来获取表格
    '''
    if isinstance(file, str):
        pages = pdfplumber.open(file).pages
    if isinstance(file, pdfplumber.pdf.PDF):
        pages = file.pages
    tables = []
    for page in pages:
        raw_tables = pdfplumber_by_line(page)
        if not raw_tables:
            raw_tables = pdfplumber_by_text(
                page, text_tolerance=2)
        if not raw_tables:
            continue
        for raw_table in raw_tables:
            raw_table_ = raw_table.extract()
            pd_table = pd.DataFrame(raw_table_)
            # 有时会将较为整齐的文本当成表格，去掉这种 1*x 或 x*1的不正确的表格
            if 1 in list(pd_table.shape):
                continue
            tables.append(pd_table
                # 'bbox':raw_table.bbox
            )
    return tables

tables_1 = extract_table_by_pdfplumber(file_path)
print(len(tables_1));tables_1[-1]

29
CPU times: user 11.6 s, sys: 180 ms, total: 11.8 s
Wall time: 11.9 s


In [344]:
def pdfplumber_by_line(page):
    # 根据pdf中的线条来拆分表格，一般比较精准
    tables = page.find_tables(table_settings={
        "vertical_strategy": "lines", 
        "horizontal_strategy": "lines",
        "intersection_tolerance": 2,
    })
    return tables

def pdfplumber_by_text(page, text_tolerance=1):
    # 根据pdf页中的文本对齐来拆分表格，结果会更全，但不一定正确
    tables = page.find_tables(table_settings={
        "vertical_strategy": "text", 
        "horizontal_strategy": "text",
        # "intersection_tolerance": 2,
        "text_tolerance": text_tolerance,
        # "text_x_tolerance": 1, # pixels
        # "text_y_tolerance": 5,
    })
    return tables

def pdfplumber_find_title(page, bbox):
    # 根据表格的bbox位置，来查找它上面的标题
    pass

In [520]:
def process_raw_table(table, drop_column_ratio=0.1, drop_row_ratio=0.1):
    '''处理原始的表格，删除空白行等 
    '''
    if isinstance(table, list):
        table = pd.DataFrame(table)
    
    # 将 None 和 空白字符 替换成 NaN
    table = table.applymap(lambda x:x if x else np.NaN)
    # 删除全部是空或者90%是空的行、列
    # column_valid_ratio = table.count(axis=0)/table.shape[0]
    # table = table.T[column_valid_ratio > drop_column_ratio].T
    # row_valid_ratio = table.count(1)/table.shape[1]
    # table = table[row_valid_ratio > drop_row_ratio]
    
    # 删除全部是空的行和列
    table.dropna(axis=0, how='all', inplace=True)
    table.dropna(axis=1, how='all', inplace=True)
    
    # 将大量空白符以及换行符等，替换成单个空白符
    table.replace(regex=r'(：)', value=': ', inplace=True)
    # 数字里的逗号会产生干扰
    table.replace(regex=r'(,)', value='',inplace=True)
    table.replace(regex=[r'(\n)', r'(  )'], value=' ', inplace=True)
    
    return table

# process_raw_table(tables[0].extract())
# tables[0].extract()

In [502]:
# 合并不同页面的句子？
page_5 = pages[4]
sentence = [[],]
bottom = 0
for i, char in enumerate(pages[8].chars):
#     for key,val in char.items():
#         print(f"{key}:{str(val)}", end=' | ')
#     print('')
#     print(char['text'], end='')
    if float(bottom) + 200 > float(char['bottom']) > float(bottom) + 10:
        bottom = float(char['bottom'])
        sentence.append([char['text']])
    else:
        sentence[-1].append(char['text'])
    # print(i, char['text'], end=' ')
for s in sentence:
    print(''.join(s))
    #print(s)


2018年半年度报告  9 
注： 
（1）公司根据财政部《企业会计准则第42号——持有待售非流动资产、处置组和终止经营》及《企业
会计准则第16号——政府补助》的规定，将原计入在“营业外收入”和“营业外支出”中的相关资产处置
利得或损失及与企业日常活动相关的政府补助分别计入新增的“资产处置损益”及“其他收益”项目，并
对比较期相关报表项目及财务指标重述，上述规定对公司利润总额和净利润没有影响。 
（2）基本每股收益、加权平均净资产收益率根据《公开发行证券的公司信息披露编报规则第9号——净
资产收益率和每股收益的计算及披露（2010年修订）》计算。报告期基本每股收益按照发行在外的普通股
加权平均数29,352,080,397股计算得出。 
（3）2018年3月，公司对浦发优2优先股发放股息人民币8.25亿元。在计算本报告披露的每股收益及
加权平均净资产收益率等指标时，公司考虑了浦发优2优先股股息发放的影响。 
（4）非经常性损益根据《中国证券监督管理委员会公告2008年第43号—公开发行证券的公司信息披露
解释性公告第1号——非经常性损益》的定义计算。 
（5）平均总资产收益率=净利润/资产平均余额，资产平均余额＝（期初资产总额＋期末资产总额）/2。 
（6）全面摊薄净资产收益率=报告期归属于母公司普通股股东的净利润/期末归属于母公司普通股股东的
净资产。 
（7）扣除非经常性损益后全面摊薄净资产收益率=报告期归属于母公司普通股股东的扣除非经常性损益
的净利润/期末归属于母公司普通股股东的净资产。 
（8）净利差为总生息资产平均收益率与总计息负债平均成本率两者的差额。净利息收益率=利息净收入/
总生息资产平均余额。根据2018年1月1日实行的新金融工具准则的要求，报告期内以公允价值计量且其变
动计入当期损益的金融资产产生的利息计入投资损益，若按原口径还原至利息收入，集团净利差为1.72%，
净利息收益率为1.81%。 
（9）成本收入比=业务及管理费/营业收入。 
（10）报告期加权平均净资产收益率、扣除非经常性损益后的加权平均净资产收益率、全面摊薄净资产
收益率、扣除非经常性损益后全面摊薄净资产收益率、平均总资产收益率未年化处理。 
2.2企业会计准则与国际财务报告准则下会计数据差异 
按照企业会计准则编制的财务报表和按照国际财务报告准则编制的财务报表中列示的


In [683]:
pd.DataFrame.to_csv?

Signature: pd.DataFrame.to_csv(self, path_or_buf=None, sep=',', na_rep='', float_format=None, columns=None, header=True, index=True, index_label=None, mode='w', encoding=None, compression=None, quoting=None, quotechar='"', line_terminator='\n', chunksize=None, tupleize_cols=None, date_format=None, doublequote=True, escapechar=None, decimal='.')
Docstring:
Write DataFrame to a comma-separated values (csv) file

Parameters
----------
path_or_buf : string or file handle, default None
    File path or object, if None is provided the result is returned as
    a string.
sep : character, default ','
    Field delimiter for the output file.
na_rep : string, default ''
    Missing data representation
float_format : string, default None
    Format string for floating point numbers
columns : sequence, optional
    Columns to write
header : boolean or list of string, default True
    Write out the column names. If a list of strings is given it is
    assumed to be aliases for the column names
inde

In [542]:
excel_path = file_path.replace('.PDF','').replace('.pdf', '')
with pd.ExcelWriter(excel_path + '.xlsx') as excel:
    for i, table in enumerate(ttt):
        print(i+1,end=' ')
        #table.to_excel(excel, f'Sheet{i+1}')

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 